1. Jelenítsük meg minden egyes szoba esetén, hogy hányszor foglalták le!
    
    - Csak a szoba azonosítója és a foglalások darabszáma jelenjen meg
    - Azok a szobák is jelenjenek meg, ahol nem történt foglalás

In [ ]:
SELECT 
    sz.szoba_id AS 'Azonosító',
    COUNT(f.foglalas_pk) AS 'Foglalások száma'
FROM szoba sz LEFT JOIN Foglalas f ON sz.szoba_id = f.szoba_fk
GROUP BY sz.szoba_id
ORDER BY [Foglalások száma]


2. <span style="color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px; background-color: rgb(255, 255, 255);">Listázzuk azon szálláshelyek azonosítóját és nevét, amelyek a saját helyükön (HELY) a legtöbb csillaggal rendelkeznek!</span>

In [ ]:
SELECT
    sz.SZALLAS_ID AS "Azonosító",
    sz.SZALLAS_NEV AS "Szállás neve"
FROM Szallashely sz
WHERE sz.CSILLAGOK_SZAMA = (
    SELECT MAX(szh.CSILLAGOK_SZAMA)
    FROM Szallashely szh
    WHERE sz.HELY = szh.HELY
)
ORDER BY sz.CSILLAGOK_SZAMA DESC;

3. Készítsen lekérdezést, amely sorszámozza a vendégeket a foglalások száma alapján
    
    - A sorszám mellett jelenjen meg a vendég azonosítója, neve és a foglalások száma is
    - A legtöbbet foglaló vendég kapja az egyes sorszámot
    - Holtverseny esetén a számozás annyit ugorjon, ahány egyforma érték van

In [ ]:
SELECT
    RANK() OVER (ORDER BY COUNT(f.foglalas_pk) DESC) AS "Sorszám",
    v.usernev AS "Azonosító",
    v.nev AS "Név",
    COUNT(f.foglalas_pk) AS "Foglalások száma"
FROM Vendeg v LEFT JOIN Foglalas f ON v.USERNEV = f.UGYFEL_FK
GROUP BY v.usernev, v.nev

4. Jelenítsük meg, hogy hány foglalás történt szobánként, naponta (METTOL), valamint szobánként, azon belül naponta 
    
    - Szempontonként külön-külön számoljuk a foglalásokat
    - A foglalások száma minden esetben legalább 2 legyen

5. Készítsünk ideiglenes táblát #MASKED\_GUEST néven, ahol 
    
    - a tábla felépítése megegyezik a vendég tábláéval, DE
    - a vendégek nevének csak az első 2, és az utolsó 3 karaktere jelenjen meg, közötte X-ek legyenek!

In [ ]:
CREATE TABLE #MASKED_GUEST(
    usernev nvarchar(20) PRIMARY KEY,
    nev nvarchar(20) MASKED WITH (FUNCTION = 'partial(2, "XXXX", 3)'),
    email nvarchar(60),
    szaml_cim nvarchar(100),
    szul_dat date,
)

In [ ]:
SELECT * FROM Vendeg


6\. Valósítsa meg REDIS-ben a következő táblákat:

- <span style="box-sizing: border-box; font-size: 0.9375rem;">IRO(ID, nev, nemzetiseg)</span>
- KONYV(ID, cim, mufaj, IRO\_ID)

Vigyen fel táblánként legalább 2-2 sornyi adatot!

HSET IRO:1 ID 1 nev "Vámos Miklós" nemzetiseg "magyar"

HSET IRO:2 ID 2 nev "Kepes András" nemzetiseg "magyar"

HSET KONYV:1 ID 1 cim "Hattyúk dala" mufaj "regény" IRO\_ID 1

HSET KONYV:2 ID 2 cim "Igen" mufaj "elbeszélés" IRO\_ID 1

HSET KONYV:3 ID 3 cim "A boldog hülye és az okos depressziós" mufaj "elbeszélés" IRO\_ID 2

HSET KONYV:4 ID 4 cim "Világkép" mufaj "elbeszélés" IRO\_ID 2

7. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Cassandra CQL Shellben hozza létre a következő táblát:</span>

STUDENT(ID, Nev, Teljesitett\_felevek, Eredmenyek, Nyelvtudas)

- Az ID legyen az elsődleges kulcs
- A Teljesitett félévek legyen LIST, tartalma egész számok, pl: 1, 2, 3
- Az Eredmenyek legyen MAP, tartalma, hogy melyik félévben mennyi volt az átlag, pl: 1: 3,00, 2: 4.55, 3: 4,10
- A Nyelvtudas legyen SET, tartalma a nyelvek megnevezése

Vigyen fel legalább 2 rekordot!

8. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">A Neo4j Sandbox-ban vagy a Desktopban jelenítsük meg azon személyeket, akik egy filmben szerepeltek Tom Hanks-sel.</span>

- Egy szereplő neve csak egyszer jelenjen meg
- Tom Hanks ne legyen benne a listában

MATCH (p:Person {name: 'Tom Hanks'})-\[:ACTED\_IN\]-\>(movie:Movie)\<-\[:ACTED\_IN\]-(p2:Person)

WHERE p.name \<\> p2.name

RETURN DISTINCT p2.name AS Actor

ORDER BY Actor;